In [ ]:
# 버젼이 달라지면 패키지 불러오는 방식이 달라집니다.
# 반드시 아래 버젼으로 설치해주세요
# pip install torchtext==0.10.0

In [ ]:
# torchtext 관련 패키지 불러오기
import torch
from torchtext.legacy import data
from torchtext.legacy.data import Dataset, Example, BucketIterator

In [ ]:
# 질의 응답 생성을 위한 샘플 데이터 셋 생성
# 데이터 구성 [(질문, 답변), (질문, 답변), ...]
QA = [
    ("What is the primary function of the heart?", "To pump blood through the circulatory system."),
    ("Where is the Great Barrier Reef situated?", "Off the coast of Queensland, Australia"),
    ("What is the Sun mainly composed of?", "Hot plasma"),
    ("Who designed the Eiffel Tower?", "Gustave Eiffel"),
    ("Will there be a London Marathon in October 2023?", "In August 2021, race organisers confirmed that the 2023 event would take place on 23 April;")
]

In [ ]:
# Field 메서드를 활용하여 각 데이터에 맞는 전처리 방식 설정

# 질문데이터 Field 구성
QPREPROCESS = data.Field(
    # 띄어쓰기 기준으로 토큰화
    tokenize=lambda x: x.split(),
    # 배치차원을 첫번째 순서로
    batch_first=True,
    # (미니배치, 문장길이) 형태로 반환
    include_lengths=True,
    # torch.long 타입으로 반환
    dtype=torch.long
)

# 답변데이터 Field 구성
APREPROCESS = data.Field(
    # 띄어쓰기 기준으로 토큰화
    tokenize=lambda x: x.split(),
    # 배치차원을 첫번째 순서로
    batch_first=True,
    # (미니배치, 배치모양) 형태로 반환
    include_lengths=True,
    # 필요하면 리턴 길이 고정
    #fix_length=10,
    # bos 토큰 설정
    init_token='<BOS>',
    # eos 토큰 설정
    eos_token='<EOS>',
    # torch.long 타입으로 반환
    dtype=torch.long
)

In [ ]:
# 예시 데이터 생성
example = []
for qa_tuple in QA:
    input_data = Example.fromlist(
        qa_tuple,
        fields=[('question', QPREPROCESS),('answer', APREPROCESS)]
    )
    example.append(input_data)

In [ ]:
# torchtext dataset 구축 (example 리스트 사용)
dataset = Dataset(
    example,
    fields=[('question', QPREPROCESS),('answer', APREPROCESS)]
)

In [ ]:
# BucketIterator 생성(파이토치 데이터 로더 역할 수행)
iterator = BucketIterator(dataset, batch_size=1, sort_key=lambda x: len(x.text))

In [ ]:
# 어휘 구축(단어 : 인덱스 구축을 위해 반드시 수행해야합니다 !)
QPREPROCESS.build_vocab(dataset)
APREPROCESS.build_vocab(dataset)

In [ ]:
#학습데이터 확인

In [ ]:
#batch_size=1 일 때

In [ ]:
sample = next(iter(iterator))

# 질문 데이터 확인
sample.question[0]

# 질문 데이터 모양 확인
sample.question[1]  #만약, include_lengths=True일때만 확인 가능

# 답변 데이터 확인
sample.answer[0]

# 답변 데이터 모양확인
sample.answer[1]

In [ ]:
#batch_size 가 2 이상일 때

In [ ]:
# batch_size가 2 이상인 BucketIterator 생성(파이토치 데이터 로더 역할 수행)
iterator = BucketIterator(dataset, batch_size=2, sort_key=lambda x: len(x.text))

sample = next(iter(iterator))

# 질문 데이터 확인
sample.question[0] # 두번째 질문끝에 1로 패딩되어 길이가 고정되어 있음을 확인할 수 있음

# 질문 데이터 모양 확인
sample.question[1]  #패딩을 제외한 길이로 표시됨

# 답변 데이터 확인
# 답변데이터 셋은 <BOS>가 2로 <EOS> 3으로 구성되어 있음을 확인할 수 있음
sample.answer[0]

sample.answer[1] #패딩을 제외한 길이로 표시됨

In [ ]:
#전처리를 위한 단어 인덱스 사전 확인

In [ ]:
# 질문 데이터 단어사전확인
QPREPROCESS.vocab.stoi

In [ ]:
# 답변 데이터 단어사전확인
APREPROCESS.vocab.stoi